In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
from interlab.queries import query_for_json, query_model
import langchain_community.chat_models
from treetrace import TracingNode
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
from interlab_zoo.resolve.resolve import Resolver

import langchain
import xml.etree.ElementTree as ET

dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [5]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
set_llm_cache(InMemoryCache())


In [6]:
factory = Factory.from_yaml("audience.yaml")

In [8]:

def resolve_direct(resolver, channel=ChatChannel, max_steps_per_player=7):
    players = resolver.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [resolver.judge])
        if resolver.agreement_check():
            return


def resolve_with_mediator(sitation, mediator_key, channel=ChatChannel, max_steps_per_player=7):
    names = and_join(p.name for p in resolver.players)
    mediator = resolver.factory.create_persona(
        mediator_key,
        public=f"$name is a mediator to arrange arrange an agreement between {names}",
        goal=f"to help {names} find an agreement on {resolver.subject}; you will not watch the movie")
    players = resolver.players
    link(players + [mediator])
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message(sender, mediator, others + [resolver.judge])
        channel.query_and_send_message_to_all(mediator, players, [resolver.judge])
        if resolver.agreement_check():
            return


with TracingNode("root") as root:
    resolver = Resolver("a specific movie name they want to watch together", factory, ["Maria-gpt35", "Connor-gpt35"])

    with TracingNode("resolving"):
        resolve_with_mediator(resolver, "Edward-gpt35")
    print(resolver.agreement_result)
    with TracingNode("scoring"):
        scoring = resolver.score_result()
    df = pd.DataFrame(scoring, columns=["key", "result", "score_basic", "score_wh"])
    print(df)
    
root.display()

/home/ada/.cache/pypoetry/virtualenvs/interlab-cZHazbI_-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The Grand Budapest Hotel
            key                    result  score_basic  score_wh
0   Maria-gpt35  The Grand Budapest Hotel          9.0       9.0
1  Connor-gpt35  The Grand Budapest Hotel          8.5       9.0


In [18]:
root.display()